# Global

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/MyDrive/Try_Robotics2_project/"
except:
    root_path = "./"

In [ ]:
global opt_update, get_params, nn_forward_fn
global best_params, best_loss

# Arguments encoder
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d


# Imports

In [ ]:
import argparse
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl

from copy import deepcopy as copy
from functools import partial
from jax import jit
from jax import random
from jax.example_libraries import stax
from jax.example_libraries.stax import serial, Dense, Softplus, Tanh, elementwise, Relu
from jax.example_libraries import optimizers
from jax.experimental.ode import odeint
from jax.tree_util import tree_flatten
from tqdm import tqdm

# Utils

In [ ]:
def wrap_coords(state):
  return jnp.concatenate([(state[:2] + jnp.pi) % (2 * jnp.pi) - jnp.pi, state[2:]])

def rk4_step(f, x, t, h):
  k1 = h * f(x, t)
  k2 = h * f(x + k1/2, t + h/2)
  k3 = h * f(x + k2/2, t + h/2)
  k4 = h * f(x + k3, t + h)
  return x + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

def radial2cartesian(t1, t2, l1, l2):
  x1 = l1 * jnp.sin(t1)
  y1 = -l1 * jnp.cos(t1)
  x2 = x1 + l2 * jnp.sin(t2)
  y2 = y1 - l2 * jnp.cos(t2)
  return x1, y1, x2, y2

def write_to(data, path):
  with open(path, 'wb') as f:
    pkl.dump(data, f, protocol=pkl.HIGHEST_PROTOCOL)

def read_from(path):
  with open(path, 'rb') as f:
    data = pkl.load(f)
  return data

def plot_loss(train_losses, test_losses):
  fig, ax = plt.subplots(1, 1)

  ax.plot(train_losses, label='Train loss')
  ax.plot(test_losses, label='Test loss')
  ax.set_ylabel("Loss")
  ax.set_xlabel("Epochs")
  ax.legend()

  fig.tight_layout()

# Lagrangian Dynamics

In [ ]:
def unconstrained_eom(model, state, t=None):
  q, q_t = jnp.split(state, 2)
  return jnp.concatenate([q_t,model(q, q_t).reshape([2])])

def lagrangian_eom(lagrangian, state, t=None):
  q, q_t = jnp.split(state, 2)
  q = q % (2*jnp.pi)
  q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
          @ (jax.grad(lagrangian, 0)(q, q_t)
             - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
  dt = 1e-1
  return dt*jnp.concatenate([q_t, q_tt])

def raw_lagrangian_eom(lagrangian, state, t=None):
  q, q_t = jnp.split(state, 2)
  q = q % (2*jnp.pi)
  q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
          @ (jax.grad(lagrangian, 0)(q, q_t)
             - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
  return jnp.concatenate([q_t, q_tt])

def lagrangian_eom_rk4(lagrangian, state, n_updates, Dt=1e-1, t=None):
    @jax.jit
    def cur_fnc(state):
        q, q_t = jnp.split(state, 2)
        q = q % (2*jnp.pi)
        q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
                 @ (jax.grad(lagrangian, 0)(q, q_t)
                 - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
        return jnp.concatenate([q_t, q_tt])

    @jax.jit
    def get_update(update):
        dt = Dt/n_updates
        cstate = state + update
        k1 = dt*cur_fnc(cstate)
        k2 = dt*cur_fnc(cstate + k1/2)
        k3 = dt*cur_fnc(cstate + k2/2)
        k4 = dt*cur_fnc(cstate + k3)
        return update + 1.0/6.0 * (k1 + 2*k2 + 2*k3 + k4)

    update = 0
    for _ in range(n_updates):
        update = get_update(update)
    return update


def solve_dynamics(dynamics_fn, initial_state, is_lagrangian=True, **kwargs):
  eom = lagrangian_eom if is_lagrangian else unconstrained_eom
  @partial(jax.jit, backend='cpu')
  def f(initial_state):
    return odeint(partial(eom, dynamics_fn), initial_state, **kwargs)
  return f(initial_state)


def custom_init(init_params, seed=0):
    import numpy as np
    new_params = []
    rng = jax.random.PRNGKey(seed)
    i = 0
    number_layers = len([0 for l1 in init_params if len(l1) != 0])
    for l1 in init_params:
        if (len(l1)) == 0: new_params.append(()); continue
        new_l1 = []
        for l2 in l1:
            if len(l2.shape) == 1:
                new_l1.append(jnp.zeros_like(l2))
            else:
                n = max(l2.shape)
                first = int(i == 0)
                last = int(i == number_layers - 1)
                mid = int((i != 0) * (i != number_layers - 1))
                mid *= i

                std = 1.0/np.sqrt(n)
                std *= 2.2*first + 0.58*mid + n*last

                if std == 0:
                    raise NotImplementedError("Wrong dimensions for MLP")

                new_l1.append(jax.random.normal(rng, l2.shape)*std)
                rng += 1
                i += 1

        new_params.append(new_l1)

    return new_params

# Physics

In [ ]:
a = 50
@jit
def test_kinetic_energy(state, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  q, q_dot = jnp.split(state,2)
  (t1, t2), (w1, w2) = q, q_dot

  T1 = 0.5 * (I+m1*(d1**2)) * (w1**2)
  T2 = 0.5 * Im * (w2**2)
  T = T1 + T2
  return T

@jit
def test_potential_energy(state, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  q, q_dot = jnp.split(state,2)
  (t1, t2), (w1, w2) = q, q_dot

  Ug = -m1*g*d1*jnp.cos(t1)
  Ue = 0.5*k*((t1-t2)**2)
  V = Ug+Ue
  return V
@jit
def kinetic_energy(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  (t1, t2), (w1, w2) = q, q_dot

  T1 = 0.5 * (I+m1*(d1**2)) * (w1**2)
  T2 = 0.5 * Im * (w2**2)
  T = T1 + T2
  return T

@jit
def potential_energy(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  (t1, t2), (w1, w2) = q, q_dot

  Ug = -m1*g*d1*jnp.cos(t1)
  Ue = 0.5*k*((t1-t2)**2)
  V = Ug+Ue
  return V

@jit
def lagrangian_fn(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  (t1, t2), (w1, w2) = q, q_dot

  T = kinetic_energy(q, q_dot)
  V = potential_energy(q, q_dot)
  return jnp.sum(T - V)

@jit
def hamiltonian_fn(q, q_dot, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  (t1, t2), (w1, w2) = q, q_dot

  T = kinetic_energy(q, q_dot)
  V = potential_energy(q, q_dot)
  return T + V

@jit
def analytical_fn(state, t=0, m1=0.25, l1=0.3, g=9.81, I=0.01125, d1=0.15, Im=0.0128,k=a):
  t1, t2, w1, w2 = state
  a1 = 0
  a2 = 0
  f1 = (-m1*g*d1*jnp.sin(t1)+k*(t1-t2))/(I+m1*(d1**2))
  f2 = k*(t1-t2)/Im
  g1 = f1
  g2 = f2
  return jnp.stack([w1, w2, g1, g2])


# Data

In [ ]:
@partial(jax.jit, backend='cpu')
def get_trajectory(y0, times, use_lagrangian=False, **kwargs):
  if use_lagrangian:
    y = solve_dynamics(lagrangian_fn, y0, t=times, is_lagrangian=True, rtol=1e-10, atol=1e-10, **kwargs)
  else:
    y = odeint(analytical_fn, y0, t=times, rtol=1e-10, atol=1e-10, **kwargs)
  return y

@partial(jax.jit, backend='cpu')
def get_trajectory_lagrangian(y0, times, **kwargs):
  return solve_dynamics(lagrangian_fn, y0, t=times, is_lagrangian=True, rtol=1e-10, atol=1e-10, **kwargs)

@partial(jax.jit, backend='cpu')
def get_trajectory_analytic(y0, times, **kwargs):
    return odeint(analytical_fn, y0, t=times, rtol=1e-10, atol=1e-10, **kwargs)

def test_dataset(rng, samples=1, t_span=[0, 10], fps=100, test_split=0.5, lookahead=1,
                    unlimited_steps=False, init_pos=2.0*np.pi, init_vel=0.1, mot_pos=2.0*np.pi, mot_vel=0.1, **kwargs):
    data = {'meta': locals()}
    vfnc = jax.jit(jax.vmap(analytical_fn))
    vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxstep=100), (0, None), 0))
    vget_unlimited = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic), (0, None), 0))

    frames = int(fps*(t_span[1]-t_span[0]))
    times = jnp.linspace(t_span[0], t_span[1], frames)
    y0 = jnp.concatenate([
        jax.random.uniform(rng,(samples,1))*init_pos,
        jax.random.uniform(rng+2,(samples,1))*mot_pos,
        jax.random.uniform(rng+1,(samples,1))*init_vel,
        jax.random.uniform(rng+3,(samples,1))*mot_vel
    ], axis=1)

    if unlimited_steps:
      y = vget(y0, times)
    else:
      y = vget_unlimited(y0, times)

    data['x'] = y[:, :-lookahead]
    data['dx'] = y[:, lookahead:] - data['x']
    data['x'] = jnp.concatenate(data['x'])
    data['dx'] = jnp.concatenate(data['dx'])
    data['t'] = jnp.tile(times[:-lookahead], (samples,))

    split_ix = int(len(data['x']) * test_split)
    split_data = {}
    for k in ['x', 'dx', 't']:
        split_data[k], split_data['test_' + k] = data[k][:split_ix], data[k][split_ix:]
    data = split_data
    return data

def noise_test_dataset(rng, samples=1, t_span=[0, 10], fps=100, test_split=0.5, lookahead=1,
                    unlimited_steps=False, init_pos=2.0*np.pi, init_vel=0.1, mot_pos=2.0*np.pi, mot_vel=0.1, **kwargs):
    data = {'meta': locals()}
    vfnc = jax.jit(jax.vmap(analytical_fn))
    vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxstep=100), (0, None), 0))
    vget_unlimited = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic), (0, None), 0))

    frames = int(fps*(t_span[1]-t_span[0]))
    times = jnp.linspace(t_span[0], t_span[1], frames)
    y0 = jnp.concatenate([
        jax.random.uniform(rng,(samples,1))*init_pos,
        jax.random.uniform(rng+2,(samples,1))*mot_pos,
        jax.random.uniform(rng+1,(samples,1))*init_vel,
        jax.random.uniform(rng+3,(samples,1))*mot_vel
    ], axis=1)

    if unlimited_steps:
      y = vget(y0, times)
    else:
      y = vget_unlimited(y0, times)

    data['x'] = y[:, :-lookahead]
    data['x'] += jax.random.uniform(rng, data['x'].shape)*np.pi/2
    data['dx'] = y[:, lookahead:] - data['x']
    data['x'] = jnp.concatenate(data['x'])
    data['dx'] = jnp.concatenate(data['dx'])
    data['t'] = jnp.tile(times[:-lookahead], (samples,))

    split_ix = int(len(data['x']) * test_split)
    split_data = {}
    for k in ['x', 'dx', 't']:
        split_data[k], split_data['test_' + k] = data[k][:split_ix], data[k][split_ix:]
    data = split_data
    return data

def energy_test_dataset(rng, samples=1, t_span=[0, 10], fps=100, test_split=0.5, lookahead=1,
                    unlimited_steps=False, init_pos=2.0*np.pi, init_vel=0.1, mot_pos=2.0*np.pi, mot_vel=0.1, **kwargs):
    data = {'meta': locals()}

    vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxstep=100), (0, None), 0))
    vget_unlimited = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic), (0, None), 0))
    venergy = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_lagrangian), (0, None), 0))

    frames = int(fps*(t_span[1]-t_span[0]))
    times = jnp.linspace(t_span[0], t_span[1], frames)
    y0 = jnp.concatenate([
        jax.random.uniform(rng,(samples,1))*init_pos,
        jax.random.uniform(rng+2,(samples,1))*mot_pos,
        jax.random.uniform(rng+1,(samples,1))*init_vel,
        jax.random.uniform(rng+3,(samples,1))*mot_vel
    ], axis=1)

    if unlimited_steps:
        y = vget(y0, times)
    else:
        y = venergy(y0,times)

    data['x'] = y[:, :-lookahead]
    data['dx'] = y[:, lookahead:] - data['x']
    data['x'] = jnp.concatenate(data['x'])
    data['dx'] = jnp.concatenate(data['dx'])
    data['t'] = jnp.tile(times[:-lookahead], (samples,))

    split_ix = int(len(data['x']) * test_split)
    split_data = {}
    for k in ['x', 'dx', 't']:
        split_data[k], split_data['test_' + k] = data[k][:split_ix], data[k][split_ix:]
    data = split_data
    return data

# Models

In [ ]:
sigmoid = jit(lambda x: 1/(1+jnp.exp(-x)))
swish = jit(lambda x: x/(1+jnp.exp(-x)))
relu3 = jit(lambda x: jnp.clip(x, 0.0, float('inf'))**3)
Swish = elementwise(swish)
Relu3 = elementwise(relu3)

def extended_mlp(args):
    act = {
        'softplus': [Softplus, Softplus],
        'swish': [Swish, Swish],
        'tanh': [Tanh, Tanh],
        'tanh_relu': [Tanh, Relu],
        'soft_relu': [Softplus, Relu],
        'relu_relu': [Relu, Relu],
        'relu_relu3': [Relu, Relu3],
        'relu3_relu': [Relu3, Relu],
        'relu_tanh': [Relu, Tanh],
    }[args.act]
    hidden = args.hidden_dim
    output_dim = args.output_dim
    nlayers = args.layers

    layers = []
    layers.extend([
        Dense(hidden),
        act[0]
    ])
    for _ in range(nlayers - 1):
        layers.extend([
            Dense(hidden),
            act[1]
        ])

    layers.extend([Dense(output_dim)])

    return stax.serial(*layers)

# Loss

In [ ]:
# replace the lagrangian with a parameteric model
def learned_dynamics(params):
  @jit
  def dynamics(q, q_t):
#     assert q.shape == (2,)
    state = wrap_coords(jnp.concatenate([q, q_t]))
    return jnp.squeeze(nn_forward_fn(params, state),axis=-1)
  return dynamics

def make_loss(args):
    if args.loss == 'lnn':
        @jax.jit
        def gln_loss(params, batch, l2reg):
            state, targets = batch#_rk4
            leaves, _ = tree_flatten(params)
            l2_norm = sum(jnp.vdot(param, param) for param in leaves)
            preds = jax.vmap(partial(lagrangian_eom_rk4, learned_dynamics(params), Dt=args.dt, n_updates=args.n_updates))(state)
            return jnp.sum(jnp.abs(preds - targets)) + l2reg*l2_norm/args.batch_size

    elif args.loss == 'baseline_nn':
        @jax.jit
        def gln_loss(params, batch, l2reg):
          state, targets = batch
          preds = jax.vmap(partial(unconstrained_eom, learned_dynamics(params)))(state)
          return jnp.sum(jnp.abs(preds - targets)) + l2reg/args.batch_size

    else:
        print("---------------------- SELECT A CORRECT LOSS ----------------------")
        exit(0)

    return gln_loss

# Test

In [ ]:
def test_data(args,exp_init_pos,exp_init_vel,exp_mot_pos,exp_mot_vel):
  global opt_update, get_params, nn_forward_fn

  predictions_file = root_path + "Elastic/Baseline/" + args.loss + "_" +str(round(exp_init_pos,2)) + "_" + str(round(exp_init_vel,2)) + "_elastic_predictions" #colab
  noise_predictions_file = root_path + "Elastic/Baseline/" + args.loss + "_" +str(round(exp_init_pos,2)) + "_" + str(round(exp_init_vel,2)) + "_elastic_noise_predictions"
  energy_file = root_path + "Elastic/Baseline/" + args.loss + "_" +str(round(exp_init_pos,2)) + "_" + str(round(exp_init_vel,2)) + "_elastic_energy"
  noise_energy_file = root_path + "Elastic/Baseline/" + args.loss + "_" +str(round(exp_init_pos,2)) + "_" + str(round(exp_init_vel,2)) + "_elastic_noise_energy"

  opt_init, opt_update, get_params = optimizers.adam(args.lr)
  opt_state = opt_init(best_params)

  _, nn_forward_fn = extended_mlp(args)
  model = (nn_forward_fn, best_params)

  max_t = 10
  test_set = test_dataset(jax.random.PRNGKey(2),
                                t_span=[0, max_t],
                                fps=100, test_split=1.0,
                                unlimited_steps=False,
                                init_pos=exp_init_pos,
                                init_vel=exp_init_vel,
                                mot_pos=exp_mot_pos,
                                mot_vel=exp_mot_vel)

  noise_set = noise_test_dataset(jax.random.PRNGKey(2),
                                t_span=[0, max_t],
                                fps=100, test_split=1.0,
                                unlimited_steps=False,
                                init_pos=exp_init_pos,
                                init_vel=exp_init_vel,
                                mot_pos=exp_mot_pos,
                                mot_vel=exp_mot_vel)
  
  t = test_set['x'][0, :]
  n = noise_set['x'][0, :]
  p = get_params(opt_state)

  model_predictions = jax.device_get(odeint(
    partial(raw_lagrangian_eom, learned_dynamics(p)),
    t,
    np.linspace(0, max_t, num=test_set['x'].shape[0]),
    mxstep=100))
  
  energy_model_predictions = jax.device_get(odeint(
    partial(raw_lagrangian_eom, learned_dynamics(p)),
    t,
    np.linspace(0, max_t, num=test_set['x'].shape[0]),
    mxstep=100))

  np.save(predictions_file,model_predictions)

  noise_model_predictions = jax.device_get(odeint(
    partial(raw_lagrangian_eom, learned_dynamics(p)),
    n,
    np.linspace(0, max_t, num=noise_set['x'].shape[0]),
    mxstep=100))
  
  energy_noise_model_predictions = jax.device_get(odeint(
    partial(raw_lagrangian_eom, learned_dynamics(p)),
    n,
    np.linspace(0, max_t, num=noise_set['x'].shape[0]),
    mxstep=100))

  np.save(noise_predictions_file, noise_model_predictions)

  total_predicted_energy = (
      jax.vmap(test_kinetic_energy)(energy_model_predictions[:]) + \
      jax.vmap(test_potential_energy)(energy_model_predictions[:])
  )

  np.save(energy_file, total_predicted_energy)

  total_noise_predicted_energy = (
      jax.vmap(test_kinetic_energy)(energy_noise_model_predictions[:]) + \
      jax.vmap(test_potential_energy)(energy_noise_model_predictions[:])
  )

  np.save(noise_energy_file, total_noise_predicted_energy)

init_pos = 2.0*np.pi
init_vel = 0.5
mot_pos = np.pi
mot_vel = 1

loaded_model = read_from("/content/drive/MyDrive/Try_Robotics2_project/Elastic/Baseline/baseline_nn_new_elastic_params.pkl")
args = loaded_model['args']
best_params = loaded_model['params']
test_data(args,init_pos,init_vel,mot_pos,mot_vel)